In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Bidirectional 
from keras import optimizers


df = pd.read_excel('/content/drive/MyDrive/python/ind_data.xlsx')
 
df1 = pd.read_excel('/content/drive/MyDrive/python/ind_op.xlsx')



# import original input data
ip = df.values


a = df[0:1138]
b = df[1138: 2276]
o = np.stack((a,b))
for m in range(49):

    o = np.concatenate((o, [df[1138*(m+2):1138*(m+3)]]))



#import output data
y = df1[['R1 ', 'R2 ','C ', 'L ', 'k' ]]


#defining the saclers 
scale_ip = MinMaxScaler()
scale_op = MinMaxScaler()
scale_test = MinMaxScaler()


#Normalize

tt = scale_ip.fit_transform(ip)


#Split into train 

a_norm = tt[0:1138]
b_norm = tt[1138: 2276]
o_norm = np.stack((a_norm,b_norm))
all = o_norm

for m in range(49):
    o_norm = np.concatenate((o_norm, [tt[1138*(m+2):1138*(m+3)]]))

for m in range(55):
    all = np.concatenate((all, [tt[1138*(m+2):1138*(m+3)]]))

#Split into test 
p = all[51]
q = all[52]
r_new = np.stack((p,q))

for m in range(4):
    gh = all[m+53]
    r_new = np.concatenate((r_new, gh.reshape(1, 1138, 8)))


#importing Normalized input data
y_norm = scale_op.fit_transform(y)



x_norm = o_norm
x_test = r_new


x_norm = np.asarray(x_norm).astype('float32')
y_norm = np.asarray(y_norm).astype('float32')


#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)



In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')

history = model.fit(x_norm, y_norm, epochs = 100, validation_split=0.2, verbose=1)


In [ ]:
test_input = x_test
test_input = test_input.reshape(x_test.shape)
test_output = model.predict(test_input, verbose=0)

pred = scale_op.inverse_transform(test_output)
print(pred)
df5 = pd.read_excel('/content/drive/MyDrive/python/org_metrics_ind.xlsx')
y_test = df5[['R1 ', 'R2 ','C ', 'L ', 'k' ]]
org = df5.values
print('R2 for :', r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))